# Многошаговые методы

## Базовый уровень
Реализовать неявные формулы дифференцирования назад:
$$
\sum_{k = 0}^{size - 1} \alpha_k y_{i - k + 1} = h f(t_{i + 1}, y_{i + 1})
$$
Внимание! Указанные сигнатуры являются рекомендованными. Можно реализовывать иначе.
При яром нежелании писать на С++ можно писать на другом языке.


Хранение коэффициентов многошагового метода:
```c++
/* Это коэффициенты для метода на 4 шагах. Я их не заполнил */
struct BDF4{
    static constexpr unsigned int size = 4;
    static constexpr std::array<double, size> alpha = {};
};
```

Реализация класса правой части дифференциального уравнения.
То есть класс ```f(t, y)``` для ```y' = f(t,y)```.
Здесь написан пример для уравнения осциллятора ```(x, v)' = (v, -x)```
```c++

class Oscillator {
    
public:
    
    static constexpr unsigned int dim = 2;  // размерность задачи
    
    using Argument = double;  // тип аргумента, тип t
    
    using State = Eigen::Vector<double, dim>;  // состояние
    
    struct StateAndArg{
        State state;
        Argument arg;
    };

    /*** Вычисляет разницу двух состояний (решения нелиненого уравнения) ***/
    double calcDif(const State& first, const State& second)
    
    /*** Вычисляет правую часть ДУ - функцию f***/
    Eigen::Vector<double, dim> calc(const StateAndArg& stateAndArg) const {
        return Eigen::Vector<double, dim>{stateAndArg.state(1), -stateAndArg.state(0)};
    } 
};
```

Сигнатура для метода интегрирования:

```c++

struct IntegrationParameters{
    double step;  // шаг интегрирования
    double epsilon;  // точность решения нелинейного уравнения
    double maxIter;  // максимальное количество итераций для решения нелинейного уравнения
};

/***
BDF - структура с коэффициентами метода
RHS - правая часть Д.У.
RKTable -  таблица Бутчера метода Рунге-Кутты для разгона.
***/

template<typename BDF, typename RHS, template RKTable>  // таблица бутчера и класс правой части f
std::vector<typename RHS::StateAndArg> integrate(
    const typename RHS::StateAndArg& initialState,
    const typename RHS::Argument& endTime,
    const IntegrationParameters& parameters,
    const RHS& rhs
);
```
Задание:
* Реализовать интегрирование для задач Коши: ```y' = t^3, y(0) = 0```, ```y'' = - y, y(0) = 0, y'(0) = 1```.
* Провести интегрирование на отрезке ```[0, 5]``` при помощи ФДН 4 порядка 
* Построить зависимость ошибки (максимальная раность по всем точкам) от шага интегрирования в логарифмическом масштабе. Вычислить угол наклона прямой.

## Повышенный уровень

Произвести интегрирование движения в нецентральном поле Земли. Рассмотреть задачу:
$$
\textbf{r}' = \textbf{v},~~ \textbf{v} = \textbf{a}, ~~\textbf{r}(0) = \textbf{r}_0, ~~ \textbf{v}(0) = \textbf{v}_0, 
$$

Начальные условия выбрать для круговой орбиты. В качестве усорения рассмотреть:
$$
\textbf{a} = \nabla U = \nabla \left( \frac{\mu}{r} - \frac{\epsilon}{3 r^3} \left[3 \frac{z^2}{r^2} - 1 \right]\right)
$$
Формула из https://www.researchgate.net/profile/Michael-Ovchinnikov/publication/317312488_Vvedenie_v_dinamiku_kosmiceskogo_poleta/links/593161d845851553b692720b/Vvedenie-v-dinamiku-kosmiceskogo-poleta.pdf, страница 114.